# Display the evaluation information with your saved runs

You can test your own codes by adding a folder in saved folder containing real.tsv and pred.tsv
the tsv format is:
```
filename    onset   offset     event_label
```
Then by running the following cell you will see your folder

For demo some saved data are presented in the saved folder

# the following code will display the results on AR (on the similar dataset to the paper) 
The result maybe a bit different from the result on the paper because in the paper it has been run several time and the result here is only one run. If you want to see full result please run the pipeline folder

In [1]:
import os
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import ar_mme_eval.multi_eval
import ar_mme_eval.ui as ui

rootFolder='saved'
debug=0 # to show debug information
args={
    'theta_tp':0,
    'theta_fp':1,
    'beta_s':2,
    'beta_e':2,
    'show_sed':False,
    'plot_ward':False,
    'show tp/fp/fn':False
}


@interact
def result_selector(file=sorted([f for f in os.listdir(rootFolder) if os.path.isfile(f"{rootFolder}/{f}/real.tsv")])):
    if(file==None):return
    
    folder=f'{rootFolder}/{file}'
    gtf=f'{folder}/real.tsv'
    global groundtruth
    groundtruth=pd.read_csv(gtf,delimiter='\t')
    allClass=list(groundtruth['event_label'].unique()); allClass.sort();  allClass.append('macro-avg')
    if 'None' in allClass: allClass.remove('None');allClass.append('None')
    info_file='/tmp/metric_info.pkl';  info={}
    import compress_pickle
    if (os.path.exists(info_file)): info = compress_pickle.load(info_file)
    print('Please input a Display name for each run')
    runs_widget={}
    for runfile in sorted(os.listdir(f'{folder}')):
        if runfile=='real.tsv':continue
        run=runfile.replace('.tsv','')
        runs_widget[run]=widgets.Text(info.get(file+run,run),description=run)
        display(widgets.HBox([runs_widget[run],widgets.Label(run)]))
    ui.display_dataset_info(groundtruth)
    @interact_manual
    def result_selector(claz=widgets.SelectMultiple(options=allClass, description='classes')):
        compress_pickle.dump({**info,**{file+r:runs_widget[r].value  for r in runs_widget}}, info_file)
        global total_dic
        if 10:
            total_dic={}
            for run in runs_widget:
                print(f'')
                print(f'    {run}')
                pef = f'{folder}/{run}.tsv'
                print(pef)
                displayName=runs_widget[run].value
                if(displayName==''):continue
                try:
                    res1=ar_mme_eval.multi_eval.get_single_result(gtf,pef,None,debug=debug,args=args)
                    if(displayName in total_dic): print(f"Warning! a name you have selected is repeated {run}:{displayName}")
                    total_dic[displayName]=res1
                except Exception as e:
                    print('Error! submission is ignored',e)
                    raise 

        runs=list(total_dic.keys());runs.sort()
        props=list(total_dic[runs[0]].keys());#props.sort()
        metric_groups={g:[p.split(": ")[1] for p in props if g in p] for g in np.unique([p.split(": ")[0] for p in props])}
        
#         claz=['Take_Medicine']
        with pd.ExcelWriter('output.xlsx') as writer:  
            for g in metric_groups:
                fs={c:pd.concat({run:pd.concat({prop:total_dic[run][g+": "+prop].loc[c] for prop in metric_groups[g]},axis=1).T for run in runs},axis=1) for c in claz} 
                global f1
                f1=(pd.concat(fs,axis=1))
                f1.to_excel(writer, sheet_name=g)
                print(f'=============={g} Metrics ==================')
                f1.index.name=g
                
                if(g=='Our'):
                    props_short= {'detection-pakdd':'D', 'uniformity-pakdd':'U', 'total duration':'T',  'relative duration-pakdd':'R', 'boundary start-pakdd':'BS','boundary end-pakdd':'BE'}
                    display(f1.loc[[p for p in props_short]])
                    fs2={c:{run:{props_short[prop]:f1[c,run].loc[prop]['f1'] for prop in props_short} for run in runs} for c in claz}
                    ui.plot_multi_spider(fs2)
                    
                else:
                    display(f1)




interactive(children=(Dropdown(description='file', options=('0-HHMM--Home1', '0-HHMM--Home2', '0-wardpaper', '…